# Creating a database with sqlalchemy and sqlite

Quick demonstration of how to create a database to use to store datasets that are too big to store in memory. 

  I created a database and populated it from a 2.5gb csv file, so that I could run SQL queries to get small datasets into memory rather than the whole thing. (Dataset and database not included in this repository as they are too big, you can find the raw dataset [here on kaggle](https://www.kaggle.com/yingwurenjian/chicago-divvy-bicycle-sharing-data/)).
  
  This notebook has had its output cells frozen so you can still see the results of running the code, and that the finished database did actually work.


In [1]:
import pandas as pd

import warnings
warnings.filterwarnings("ignore")

import sqlite3
from sqlalchemy import create_engine

In [2]:
# Setting variables for data directory and file.
DIR = "./data"
FILE = '/data.csv'

file = '{}{}'.format(DIR, FILE)

# Quick test we have the correct path.
print('File directory: {}'.format(file))

File directory: ./data/data.csv


In [3]:
# Test data import
df = pd.read_csv(file, nrows=2)
df.head()

,trip_id,year,month,week,day,hour,usertype,gender,starttime,stoptime,...,from_station_id,from_station_name,latitude_start,longitude_start,dpcapacity_start,to_station_id,to_station_name,latitude_end,longitude_end,dpcapacity_end
0,2355134,2014,6,27,0,23,Subscriber,Male,2014-06-30 23:57:00,2014-07-01 00:07:00,...,131,Lincoln Ave & Belmont Ave,41.939365,-87.668385,15.0,303,Broadway & Cornelia Ave,41.945512,-87.645980,15.0
1,2355133,2014,6,27,0,23,Subscriber,Male,2014-06-30 23:56:00,2014-07-01 00:00:00,...,282,Halsted St & Maxwell St,41.864580,-87.646930,15.0,22,May St & Taylor St,41.869482,-87.655486,15.0


In [4]:
# Connecting to the database
csv_database = create_engine('sqlite:///csv_database.db')

In [5]:
# Building the database via chunking
chunksize = 1000000
i = 0

for df in pd.read_csv(file, chunksize = chunksize, iterator=True):
    df = df.rename(columns = {c: c.replace(' ', '') for c in df.columns})
    df.index += i
    
    df.to_sql('data', csv_database, if_exists = 'append')
    i = df.index[-1]+1
    
    print('|  index: {}'.format(i))

|  index: 1000000
|  index: 3000000
|  index: 6000000
|  index: 10000000
|  index: 15000000
|  index: 21000000
|  index: 28000000
|  index: 36000000
|  index: 45000000
|  index: 54495235


In [6]:
df = pd.read_sql_query('SELECT * FROM data WHERE from_station_id=131', csv_database)

In [7]:
df.tail()

,index,trip_id,year,month,week,day,hour,usertype,gender,starttime,...,from_station_id,from_station_name,latitude_start,longitude_start,dpcapacity_start,to_station_id,to_station_name,latitude_end,longitude_end,dpcapacity_end
19698,54492364,16739309,2017,10,39,6,13,Subscriber,Male,2017-10-01 13:04:00,...,131,Lincoln Ave & Belmont Ave,41.939365,-87.668385,15.0,214,Damen Ave & Grand Ave,41.891220,-87.676860,23.0
19699,54492992,16738014,2017,10,39,6,12,Subscriber,Female,2017-10-01 12:10:00,...,131,Lincoln Ave & Belmont Ave,41.939365,-87.668385,15.0,331,Halsted St & Blackhawk St (*),41.908537,-87.648627,20.0
19700,54492997,16738009,2017,10,39,6,12,Subscriber,Male,2017-10-01 12:09:00,...,131,Lincoln Ave & Belmont Ave,41.939365,-87.668385,15.0,331,Halsted St & Blackhawk St (*),41.908537,-87.648627,20.0
19701,54493266,16737438,2017,10,39,6,11,Subscriber,Female,2017-10-01 11:42:00,...,131,Lincoln Ave & Belmont Ave,41.939365,-87.668385,15.0,141,Clark St & Lincoln Ave,41.915689,-87.634600,23.0
19702,54494157,16735795,2017,10,39,6,10,Subscriber,Female,2017-10-01 10:06:00,...,131,Lincoln Ave & Belmont Ave,41.939365,-87.668385,15.0,156,Clark St & Wellington Ave,41.936497,-87.647539,15.0


In [8]:
df.count()

index                19703
trip_id              19703
year                 19703
month                19703
week                 19703
day                  19703
hour                 19703
usertype             19703
gender               19703
starttime            19703
stoptime             19703
tripduration         19703
temperature          19703
events               19703
from_station_id      19703
from_station_name    19703
latitude_start       19703
longitude_start      19703
dpcapacity_start     19703
to_station_id        19703
to_station_name      19703
latitude_end         19703
longitude_end        19703
dpcapacity_end       19703
dtype: int64